<!--BOOK_INFORMATION-->
<img align="left" style="width:80px;height:98px;padding-right:20px;" src="https://raw.githubusercontent.com/joe-papa/pytorch-book/main/files/pytorch-book-cover.jpg">

This notebook contains an excerpt from the [PyTorch Pocket Reference](http://pytorchbook.com) book by [Joe Papa](http://joepapa.ai); content is available [on GitHub](https://github.com/joe-papa/pytorch-book).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joe-papa/pytorch-book/blob/main/06_03_Hyperparameter_Tuning.ipynb)

# Chapter 6 - PyTorch Acceleration & Optimization

## Model Optimization

### Hyperparameter Tuning

In [ ]:
!pip install tensorboardX
!pip install ray

     |████████████████████████████████| 317kB 18.0MB/s 
     |████████████████████████████████| 47.5MB 63kB/s 
     |████████████████████████████████| 3.2MB 47.7MB/s 
     |████████████████████████████████| 133kB 56.8MB/s 
     |████████████████████████████████| 81kB 10.3MB/s 
     |████████████████████████████████| 1.3MB 40.5MB/s 
     |████████████████████████████████| 81kB 12.5MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 204kB 59.9MB/s 
     |████████████████████████████████| 296kB 57.9MB/s 
     |████████████████████████████████| 143kB 52.0MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-none-any.whl size=12621 sha256=253c8203f57397fee41792b082d5c76dba728faf44066c3305f47c9fe2bfd988
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
Successfully built gpustat


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, nodes_1=120, nodes_2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, nodes_1) # <1>
        self.fc2 = nn.Linear(nodes_1, nodes_2) # <2>
        self.fc3 = nn.Linear(nodes_2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
from ray import tune
import numpy as np

config = {
  "nodes_1": tune.sample_from(
      lambda _: 2 ** np.random.randint(2, 9)),
  "nodes_2": tune.sample_from(
      lambda _: 2 ** np.random.randint(2, 9)),
  "lr": tune.loguniform(1e-4, 1e-1),
  "batch_size": tune.choice([2, 4, 8, 16])  
  }

In [ ]:
import torch
import torchvision
from torchvision import transforms

def load_data(data_dir="./data"):
  train_transforms = transforms.Compose([
      transforms.RandomCrop(32, padding=4),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(
          (0.4914, 0.4822, 0.4465),
          (0.2023, 0.1994, 0.2010))])
  
  test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        (0.4914, 0.4822, 0.4465),
        (0.2023, 0.1994, 0.2010))])

  trainset = torchvision.datasets.CIFAR10(
      root=data_dir, train=True, 
      download=True, transform=train_transforms)

  testset = torchvision.datasets.CIFAR10(
      root=data_dir, train=False, 
      download=True, transform=test_transforms)

  return trainset, testset

In [ ]:
from torch import optim
from torch import nn
from torch.utils.data import random_split

def train_model(config):
  device = torch.device("cuda" if
    torch.cuda.is_available() else "cpu")

  model = Net(config['nodes_1'],
      config['nodes_2']).to(device=device) #<1>

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(),
                        lr=config['lr'],
                        momentum=0.9) #<2>

  trainset, testset = load_data()

  test_abs = int(len(trainset) * 0.8)
  train_subset, val_subset = random_split(
      trainset, 
      [test_abs, len(trainset) - test_abs])

  trainloader = torch.utils.data.DataLoader(
      train_subset,
      batch_size=int(config["batch_size"]),
      shuffle=True) # <3>

  valloader = torch.utils.data.DataLoader(
      val_subset,
      batch_size=int(config["batch_size"]),
      shuffle=True) # <3>

  for epoch in range(10):
      train_loss = 0.0
      epoch_steps = 0
      for data in trainloader:
          inputs, labels = data
          inputs = inputs.to(device)
          labels = labels.to(device)

          optimizer.zero_grad()

          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          train_loss += loss.item()

      val_loss = 0.0
      total = 0
      correct = 0
      for data in valloader:
          with torch.no_grad():
              inputs, labels = data
              inputs = inputs.to(device)
              labels = labels.to(device)

              outputs = model(inputs)
              _, predicted = torch.max(
                          outputs.data, 1)
              total += labels.size(0)
              correct += \
                (predicted == labels).sum().item()

              loss = criterion(outputs, labels)
              val_loss += loss.cpu().numpy()

      print(f'epoch: {epoch} ',
            f'train_loss: ',
            f'{train_loss/len(trainloader)}',
            f'val_loss: ',
            f'{val_loss/len(valloader)}',
            f'val_acc: {correct/total}')
      tune.report(loss=(val_loss / len(valloader)),
                  accuracy=correct / total)

In [ ]:
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=10,
    grace_period=1,
    reduction_factor=2)

reporter = CLIReporter(
    metric_columns=["loss", 
                    "accuracy", 
                    "training_iteration"])

In [ ]:
from functools import partial

result = tune.run(
    partial(train_model),
    resources_per_trial={"cpu": 2, "gpu": 1},
    config=config,
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=reporter)

2021-03-11 17:25:04,376	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
2021-03-11 17:25:07,204	WARNING experiment.py:292 -- No name detected on trainable. Using DEFAULT.
2021-03-11 17:25:07,206	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.
2021-03-11 17:25:07,208	WARNING function_runner.py:541 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


== Status ==
Memory usage on this node: 1.3/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2/2 CPUs, 1/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/DEFAULT_2021-03-11_17-25-07
Number of trials: 1/10 (1 RUNNING)
+---------------------+----------+-------+--------------+------------+-----------+-----------+
| Trial name          | status   | loc   |   batch_size |         lr |   nodes_1 |   nodes_2 |
|---------------------+----------+-------+--------------+------------+-----------+-----------|
| DEFAULT_b965e_00000 | RUNNING  |       |            8 | 0.00166288 |       256 |       256 |
+---------------------+----------+-------+--------------+------------+-----------+-----------+


(pid=215) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ./data/cifar-10-python.tar.gz


 95%|█████████▌| 162048000/170498071 [00:02<00:00, 86995559.36it/s]
170499072it [00:02, 65506953.99it/s]                               


(pid=215) Extracting ./data/cifar-10-python.tar.gz to ./data
(pid=215) Files already downloaded and verified
Result for DEFAULT_b965e_00000:
  accuracy: 0.4254
  date: 2021-03-11_17-25-52
  done: false
  experiment_id: bf4ddddc774a48a9b2664cf5751076fe
  hostname: fa4a1775811d
  iterations_since_restore: 1
  loss: 1.5670228345394135
  node_ip: 172.28.0.2
  pid: 215
  time_since_restore: 44.2513267993927
  time_this_iter_s: 44.2513267993927
  time_total_s: 44.2513267993927
  timestamp: 1615483552
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b965e_00000
  
== Status ==
Memory usage on this node: 3.0/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: -1.5670228345394135
Resources requested: 2/2 CPUs, 1/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/DEFAULT_2021-03-11_17-25-07
Number of trials: 2/10 (1 PENDING, 1 RUNNING)
+----------------

 92%|█████████▏| 156464128/170498071 [00:01<00:00, 88258549.52it/s]
170499072it [00:01, 86128012.97it/s]                               


...

(pid=421) Extracting ./data/cifar-10-python.tar.gz to ./data
(pid=421) Files already downloaded and verified
Result for DEFAULT_b965e_00008:
  accuracy: 0.0995
  date: 2021-03-11_17-36-59
  done: true
  experiment_id: 0257b1567ea0497db7dbd75d3ee5367d
  hostname: fa4a1775811d
  iterations_since_restore: 1
  loss: 2.413724879360199
  node_ip: 172.28.0.2
  pid: 421
  time_since_restore: 70.95776081085205
  time_this_iter_s: 70.95776081085205
  time_total_s: 70.95776081085205
  timestamp: 1615484219
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b965e_00008
  
(pid=421) epoch: 0  train_loss:  2.4061736549198627 val_loss:  2.413724879360199 val_acc: 0.0995
== Status ==
Memory usage on this node: 2.7/12.7 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: -1.2140959628105163 | Iter 4.000: -1.3276116551160813 | Iter 2.000: -1.5345812916874886 | Iter 1.000: -1.8008183712244035
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 ac

 92%|█████████▏| 157617152/170498071 [00:01<00:00, 91250166.99it/s]
170499072it [00:01, 97393608.54it/s]                               


(pid=445) Extracting ./data/cifar-10-python.tar.gz to ./data
(pid=445) Files already downloaded and verified
Result for DEFAULT_b965e_00009:
  accuracy: 0.3751
  date: 2021-03-11_17-38-10
  done: false
  experiment_id: 5d967723116f4ec59fd2d465d42d611e
  hostname: fa4a1775811d
  iterations_since_restore: 1
  loss: 1.6837590220183134
  node_ip: 172.28.0.2
  pid: 445
  time_since_restore: 69.59433126449585
  time_this_iter_s: 69.59433126449585
  time_total_s: 69.59433126449585
  timestamp: 1615484290
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b965e_00009
  
(pid=445) epoch: 0  train_loss:  1.8940870077863337 val_loss:  1.6837590220183134 val_acc: 0.3751
== Status ==
Memory usage on this node: 3.1/12.7 GiB
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: -1.2140959628105163 | Iter 4.000: -1.3276116551160813 | Iter 2.000: -1.5345812916874886 | Iter 1.000: -1.7739246208250523
Resources requested: 2/2 CPUs, 1/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0

2021-03-11 17:39:10,379	INFO tune.py:450 -- Total run time: 846.56 seconds (842.94 seconds for the tuning loop).


Result for DEFAULT_b965e_00009:
  accuracy: 0.4342
  date: 2021-03-11_17-39-10
  done: true
  experiment_id: 5d967723116f4ec59fd2d465d42d611e
  hostname: fa4a1775811d
  iterations_since_restore: 2
  loss: 1.557911741732061
  node_ip: 172.28.0.2
  pid: 445
  time_since_restore: 129.24746680259705
  time_this_iter_s: 59.653135538101196
  time_total_s: 129.24746680259705
  timestamp: 1615484350
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: b965e_00009
  
(pid=445) epoch: 1  train_loss:  1.5901369889194146 val_loss:  1.557911741732061 val_acc: 0.4342
== Status ==
Memory usage on this node: 2.9/12.7 GiB
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 8.000: -1.2140959628105163 | Iter 4.000: -1.3276116551160813 | Iter 2.000: -1.557911741732061 | Iter 1.000: -1.7739246208250523
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/DEFAULT_2021-03-11_17-25-07
Number of trials: 10/10 (1

In [ ]:
best_trial = result.get_best_trial(
    "loss", "min", "last")
print("Best trial config: {}".format(
    best_trial.config))
print("Best trial final validation loss:",
      "{}".format(
          best_trial.last_result["loss"]))
print("Best trial final validation accuracy:",
      "{}".format(
          best_trial.last_result["accuracy"]))

Best trial config: {'nodes_1': 256, 'nodes_2': 256, 'lr': 0.0016628841178862304, 'batch_size': 8}
Best trial final validation loss: 1.1886363194227219
Best trial final validation accuracy: 0.5855
